# Train the Neural Network

In [52]:
import numpy as np
import pandas as pd
import keras
import sklearn
import keras.utils
import sys
from keras import Sequential, regularizers
from keras.layers import Dense
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold, train_test_split
from numpy import argmax

Useful functions

In [53]:
def from_categorical(one_hot):
    return [argmax(x) for x in one_hot]

In [54]:
def binarize(labels):
    return [1 if x == 1 else 0 for x in labels]

Read in data

In [55]:
df = pd.read_csv('Seizure.csv').drop(columns=["Unnamed: 0"])

In [56]:
y_all = df["y"]
x_all = df.drop(columns=["y"])

Normalise

In [57]:
x_array = sklearn.preprocessing.scale(x_all)

Split data into train/test setes

In [58]:
x_train_split, x_test_split, y_train_split, y_test_split = train_test_split(x_array, y_all, test_size=0.33, random_state=42)

Index class labels from zero

In [59]:
# y_train_split -= 1
# y_test_split -= 1

Create the feedforward neural network

## Multi-class classification (unused)

In [60]:
# model = Sequential()
# model.add(Dense(88, 
#                 activation='sigmoid', 
#                 input_dim=178))
# model.add(Dense(88, activation='sigmoid'))
# model.add(Dense(5, activation='softmax'))
# model.compile(loss='categorical_crossentropy',
#                 optimizer=keras.optimizers.Adadelta(),
#                 metrics=['accuracy'])

In [61]:
# y_train_split_one_hot = keras.utils.to_categorical(y_train_split, num_classes=5)
# y_test_split_one_hot = keras.utils.to_categorical(y_test_split, num_classes=5)

In [62]:
# model.fit(x_train_split, y_train_split_one_hot, epochs=32, batch_size=16)

In [63]:
# y_test_split_predict = from_categorical(model.predict(x_test_split))
# acc = accuracy_score(y_test_split_predict, y_test_split)

In [64]:
# acc

## Binary Classification

In [65]:
acc = 0

Get a good random start

In [66]:
while acc < 0.971:
    model_binary = Sequential()
    model_binary.add(Dense(88, 
                    activation='sigmoid', 
                    input_dim=178,
                    use_bias=True))
    model_binary.add(Dense(2, activation='softmax', use_bias=True))
    model_binary.compile(loss='categorical_crossentropy',
                    optimizer=keras.optimizers.Adadelta(),
                    metrics=['accuracy'])
    
    y_train_split_binary = keras.utils.to_categorical(binarize(y_train_split), num_classes=2)
    y_test_split_binary = binarize(y_test_split)
    
    model_binary.fit(x_train_split, y_train_split_binary, epochs=31, verbose=0)
    
    y_test_split_predict_binary = from_categorical(model_binary.predict(x_test_split))
    acc = accuracy_score(y_test_split_predict_binary, y_test_split_binary)
    print(acc)
    cm = confusion_matrix(y_test_split_predict_binary, y_test_split_binary)

0.9754940711462451


In [67]:
model_binary.save_weights('model_binary.hdf5')

# Generate C code

### Helper functions

In [68]:
from __future__ import print_function

import h5py

def print_structure(weight_file_path):
    """
    Prints out the structure of HDF5 file.

    Args:
      weight_file_path (str) : Path to the file to analyze
    """
    f = h5py.File(weight_file_path)
    try:
        if len(f.attrs.items()):
            print("{} contains: ".format(weight_file_path))
            print("Root attributes:")
        for key, value in f.attrs.items():
            print("  {}: {}".format(key, value))

        if len(f.items())==0:
            return 

        for layer, g in f.items():
            print("  {}".format(layer))
            print("    Attributes:")
            for key, value in g.attrs.items():
                print("      {}: {}".format(key, value))

            print("    Dataset:")
            for p_name in g.keys():
                param = g[p_name]
                subkeys = param.keys()
                for k_name in param.keys():
                    print("      {}/{}: {}".format(p_name, k_name, param.get(k_name)[:]))
    finally:
        f.close()

In [69]:
def save_structure(weight_file_path):
    f = h5py.File(weight_file_path)
    weights = []
    try:
        if len(f.items())==0:
            return 

        for layer, g in f.items():
            for p_name in g.keys():
                param = g[p_name]
                subkeys = param.keys()
                for k_name in param.keys():
                    weights.append(param.get(k_name)[:])
    finally:
        f.close()
        return weights

In [70]:
def neurons(model):
    weights = model.get_weights()
    neurons = []
    for weight_matrix in weights:
        neurons.append(weight_matrix.shape[0])
    return neurons

In [71]:
def layers(model):
    return len(neurons(model))

In [72]:
def weights(model):
    weights = model.get_weights()
    return sum([np.prod(layer.shape) for layer in weights])

In [73]:
def neuron_string(first_connection, last_connection, 
                 activation_steepness=0, activation_function=6):
    string = "{" + str(first_connection) + \
    ", " + str(last_connection) + \
    ", " + str(activation_steepness) + \
    ", " + str(activation_function) + "}"
    return string

In [74]:
def gen_neuron_array_string(model):
    num_neurons = neurons(model)
    neuron_array_string = 'fann_neuron fann_neurons[{}] = '.format(sum(num_neurons)) + '{'
    for layer_index in range(len(num_neurons)):
        # Input layer
        if layer_index == 0:
            for neuron in range(num_neurons[layer_index]):
                neuron_array_string += neuron_string(0, 0, 0, 0) + ", "
            total_neurons = num_neurons[0]
        else:
            for neuron in range(num_neurons[layer_index]):
                start_neuron = total_neurons
                end_neuron = total_neurons + num_neurons[layer_index-1]
                neuron_array_string += neuron_string(start_neuron, end_neuron, 1.00, 6) + ", "
                total_neurons = end_neuron
    neuron_array_string = neuron_array_string[:-2] + '};'
    return neuron_array_string

In [75]:
def gen_layer_array_string(model):
    num_neurons = neurons(model)
    layer_array_string = 'fann_layer fann_layers[{}] = '.format(len(num_neurons)) + '{'
    total_neurons = 0
    for i in range(len(num_neurons)):
        start, end = total_neurons, total_neurons + num_neurons[i]
        layer_array_string += '{' + '{}, {}'.format(start, end) + '}, '
        total_neurons = end
    layer_array_string = layer_array_string[:-2] + '};'
    return layer_array_string

In [76]:
def gen_weight_array_string(model):
    weights = model.get_weights()
    total_weights = sum([np.prod(layer.shape) for layer in weights])
    weight_array_string = 'fann_type fann_weights[{}] = '.format(total_weights) + '{'
    for weight_matrix in weights:
        if len(weight_matrix.shape) == 1:
            for i in range(len(weight_matrix)):
                weight_array_string += str(weight_matrix[i]) + ', '
        elif len(weight_matrix.shape) == 2:
            for i in range(weight_matrix.shape[0]):
                for j in range(weight_matrix.shape[1]):
                    weight_array_string += str(weight_matrix[i][j]) + ', '
        else:
            raise Exception('Weight matrix shape is incorrect')
    weight_array_string = weight_array_string[:-2] + '};'
    return weight_array_string

## Generate fann_net.h

In [77]:
def gen_fann_net_h(model):
    try:
        num_neurons = sum(neurons(model))
        num_weights = weights(model)
        num_layers = layers(model)        
        f = open('fann_net.h', 'w')
        f.write('#ifndef FANN_FANN_NET_H_\n')
        f.write('#define FANN_FANN_NET_H_\n\n')
        f.write('#include "fann.h"\n')
        f.write('#include "fann_structs.h"\n\n')
        f.write('extern const enum fann_nettype_enum network_type;\n\n')
        f.write('extern fann_neuron fann_neurons[' + str(num_neurons) + '];\n\n')
        f.write('extern fann_type fann_weights[' + str(num_weights) + '];\n\n')
        f.write('extern fann_layer fann_layers[' + str(num_layers) + '];\n\n')
        f.write('#endif // FANN_FANN_NET_H')
    finally:
        f.close()

## Generate fann_net.c

In [78]:
def gen_fann_net_c(model):
    try:
        f = open('fann_net.c', 'w')
        f.write('#include "fann_net.h"\n\n')
        f.write('const enum fann_nettype_enum network_type = 0;\n\n')
        neuron_array_string = gen_neuron_array_string(model)
        weight_array_string = gen_weight_array_string(model) 
        layer_array_string = gen_layer_array_string(model)
        f.write(neuron_array_string + "\n\n")
        f.write(weight_array_string + "\n\n")
        f.write(layer_array_string + "\n\n")
    finally:
        f.close()

## Run Generate Functions

In [79]:
gen_fann_net_h(model_binary)

In [80]:
gen_fann_net_c(model_binary)

# Generate Test Data

In [91]:
def gen_fann_data_h(features, classes, num_samples):
    try:
        features = features[:num_samples, :]
        classes = classes[:num_samples]
        if features.shape[0] != len(classes):
            raise Exception("Number of samples do not match")
            
        num_inputs = np.product(features.shape)
        num_outputs = len(classes)
        input_array_string = 'extern fann_type test_data_input[{}];\n\n'.format(num_inputs)
        output_array_string = 'extern const int test_data_output[{}];\n\n'.format(num_outputs)        
        f = open('fann_data.h', 'w')
        f.write('#ifndef FANN_FANN_TEST_DATA_H_\n')
        f.write('#define FANN_FANN_TEST_DATA_H_\n\n')
        f.write(input_array_string)
        f.write(output_array_string)
        f.write('#endif // FANN_FANN_TEST_DATA_H_')
    finally:
        f.close()

In [92]:
def gen_fann_data_c(features, classes, num_samples):
    try:
        features = features[:num_samples, :]
        classes = classes[:num_samples]
        num_features = features.shape[1]
        if features.shape[0] != len(classes):
            raise Exception("Number of samples do not match")
            
        num_inputs = np.product(features.shape)
        num_outputs = len(classes)
        input_array_string = 'fann_type test_data_input[{}] = '.format(num_inputs) + '{'
        output_array_string = 'const int test_data_output[{}] = '.format(num_outputs) + '{'
        
        for i in range(num_samples):
            for j in range(num_features):
                input_array_string += '{}, '.format(features[i][j])
        input_array_string = input_array_string[:-2] + '};\n\n'
        
        for i in range(num_samples):
            output_array_string += '{}, '.format(classes[i])
        output_array_string = output_array_string[:-2] + '};'
            
        f = open('fann_data.c', 'w')
        f.write('#include "fann_data.h"\n\n')
        f.write(input_array_string)
        f.write(output_array_string)
    finally:
        f.close()

## Run Generate Functions

In [94]:
gen_fann_data_h(x_test_split, y_test_split_binary, 100)
gen_fann_data_c(x_test_split, y_test_split_binary, 100)